In [9]:
import pandas as pd
import numpy as np
import geopandas as gpd
from haversine import haversine, Unit
import gurobipy as gp
from gurobipy import GRB

# Load and prepare data
nta_table = pd.read_csv("data/model/nta_table.csv")
pantries_table = pd.read_csv("data/model/pantries_table.csv")

# Clean supply gap column
SUPPLY_GAP_COL = "Supply Gap (lbs.)"
nta_table[SUPPLY_GAP_COL] = nta_table[SUPPLY_GAP_COL].apply(
    lambda x: float(str(x).replace(',', ''))
)

print(f"Loaded {len(nta_table)} neighborhoods and {len(pantries_table)} pantries")

# Separate excess and deficit NTAs
EXCESS_NTA = nta_table[nta_table[SUPPLY_GAP_COL] < 0].copy()
DEFICIT_NTA = nta_table[nta_table[SUPPLY_GAP_COL] > 0].copy()

excess_nta_ids = set(EXCESS_NTA['nta2020'].astype(str).str.strip().str.upper())
deficit_nta_ids = set(DEFICIT_NTA['nta2020'].astype(str).str.strip().str.upper())

print(f"\nExcess NTAs: {len(excess_nta_ids)}")
print(f"Deficit NTAs: {len(deficit_nta_ids)}")

# Filter pantries by NTA membership
# Ensure pantries table has clean NTA codes
pantries_table['nta2020_clean'] = pantries_table['nta2020'].astype(str).str.strip().str.upper()

# Filter pantries
excess_pantries = pantries_table[
    pantries_table['nta2020_clean'].isin(excess_nta_ids)
].copy()

deficit_pantries = pantries_table[
    pantries_table['nta2020_clean'].isin(deficit_nta_ids)
].copy()

# Ensure 'id' column exists and is string
excess_pantries['id'] = excess_pantries['id'].astype(str)
deficit_pantries['id'] = deficit_pantries['id'].astype(str)

print(f"Excess pantries: {len(excess_pantries)}")
print(f"Deficit pantries: {len(deficit_pantries)}")

# Build supply and demand dictionaries using pantry IDs
# Create NTA lookup dictionary for faster access
nta_lookup = {}
for _, row in nta_table.iterrows():
    nta_code = str(row['nta2020']).strip().upper()
    nta_lookup[nta_code] = {
        'gap': float(row[SUPPLY_GAP_COL]),
        'pantry_count': int(row['pantry_count']) if pd.notna(row['pantry_count']) else 1
    }

# Build supply dictionary (keyed by pantry ID)
supply = {}
for _, row in excess_pantries.iterrows():
    pantry_id = str(row['id'])
    nta_code = row['nta2020_clean']
    
    if nta_code in nta_lookup:
        gap = nta_lookup[nta_code]['gap']
        n_pantries = max(nta_lookup[nta_code]['pantry_count'], 1)
        supply[pantry_id] = abs(gap) / n_pantries if gap < 0 else 0.0
    else:
        supply[pantry_id] = 0.0

# Build demand dictionary (keyed by pantry ID)
demand = {}
for _, row in deficit_pantries.iterrows():
    pantry_id = str(row['id'])
    nta_code = row['nta2020_clean']
    
    if nta_code in nta_lookup:
        gap = nta_lookup[nta_code]['gap']
        n_pantries = max(nta_lookup[nta_code]['pantry_count'], 1)
        demand[pantry_id] = gap / n_pantries if gap > 0 else 0.0
    else:
        demand[pantry_id] = 0.0

print(f"\nSupply pantries with data: {len(supply)}")
print(f"Demand pantries with data: {len(demand)}")
print(f"Total supply (lbs): {sum(supply.values()):,.2f}")
print(f"Total demand (lbs): {sum(demand.values()):,.2f}")

# Compute distance matrix using pantry IDs
def parse_point(geom_str):
    """Extract (lat, lon) from POINT string"""
    if pd.isna(geom_str):
        return None
    try:
        # Handle "POINT (lon lat)" format
        coords = str(geom_str).replace('POINT (', '').replace(')', '').split()
        return (float(coords[1]), float(coords[0]))  # (lat, lon)
    except:
        return None

excess_pantries['coords'] = excess_pantries['geometry'].apply(parse_point)
deficit_pantries['coords'] = deficit_pantries['geometry'].apply(parse_point)

# Remove any pantries with missing coordinates
excess_pantries = excess_pantries.dropna(subset=['coords'])
deficit_pantries = deficit_pantries.dropna(subset=['coords'])

print(f"Excess pantries with valid coords: {len(excess_pantries)}")
print(f"Deficit pantries with valid coords: {len(deficit_pantries)}")

# Get lists of IDs in order
excess_ids = excess_pantries['id'].tolist()
deficit_ids = deficit_pantries['id'].tolist()

# Compute distance matrix
print("\nComputing distance matrix...")
dist_matrix = np.zeros((len(excess_ids), len(deficit_ids)))

for i, excess_id in enumerate(excess_ids):
    if i % 50 == 0:
        print(f"  Processing excess pantry {i+1}/{len(excess_ids)}")
    
    excess_coord = excess_pantries[excess_pantries['id'] == excess_id]['coords'].iloc[0]
    for j, deficit_id in enumerate(deficit_ids):
        deficit_coord = deficit_pantries[deficit_pantries['id'] == deficit_id]['coords'].iloc[0]
        dist_matrix[i, j] = haversine(excess_coord, deficit_coord, unit=Unit.MILES)

# Create DataFrame with pantry IDs as index/columns
dist_df_pantry = pd.DataFrame(
    dist_matrix,
    index=excess_ids,
    columns=deficit_ids
)

print(f"\nDistance matrix shape: {dist_df_pantry.shape}")
print(f"Distance range: {dist_matrix.min():.2f} to {dist_matrix.max():.2f} miles")

# Verify alignment and clean data
# Update supply/demand dicts to only include pantries in distance matrix
supply = {pid: supply[pid] for pid in excess_ids if pid in supply}
demand = {pid: demand[pid] for pid in deficit_ids if pid in demand}

# Remove pantries with zero supply or demand (optional - helps reduce problem size)
supply = {k: v for k, v in supply.items() if v > 0}
demand = {k: v for k, v in demand.items() if v > 0}

# Update excess_ids and deficit_ids to match
excess_ids = [pid for pid in excess_ids if pid in supply]
deficit_ids = [pid for pid in deficit_ids if pid in demand]

# Filter distance matrix
dist_df_pantry = dist_df_pantry.loc[excess_ids, deficit_ids]

print(f"Supply dict keys: {len(supply)}")
print(f"Demand dict keys: {len(demand)}")
print(f"Distance matrix rows (excess): {dist_df_pantry.shape[0]}")
print(f"Distance matrix columns (deficit): {dist_df_pantry.shape[1]}")
print(f"All supply keys in distance index: {set(supply.keys()) == set(dist_df_pantry.index)}")
print(f"All demand keys in distance columns: {set(demand.keys()) == set(dist_df_pantry.columns)}")
print(f"\nTotal supply available: {sum(supply.values()):,.2f} lbs")
print(f"Total demand needed: {sum(demand.values()):,.2f} lbs")
print(f"Supply/Demand ratio: {sum(supply.values())/sum(demand.values()):.2%}")

# OPTIMIZATION MODEL
print("\n" + "="*60)
print("BUILDING OPTIMIZATION MODEL")
print("="*60)

LAMBDA_UNMET = 1
WEIGHT_UNMET = 1.0

# Build model
model = gp.Model("pantry_rebalance")
model.setParam("OutputFlag", 1)

# Decision variables
x = {(i, j): model.addVar(lb=0.0, name=f"x_{i}_{j}") 
     for i in excess_ids for j in deficit_ids}

u = {j: model.addVar(lb=0.0, name=f"u_{j}") for j in deficit_ids}

model.update()

print(f"\nVariables created:")
print(f"  Flow variables (x): {len(x):,}")
print(f"  Unmet variables (u): {len(u):,}")

# Objective function
transport_cost = gp.quicksum(
    dist_df_pantry.loc[i, j] * x[i, j]
    for i in excess_ids for j in deficit_ids
)

unmet_cost = LAMBDA_UNMET * gp.quicksum(
    WEIGHT_UNMET * u[j] for j in deficit_ids
)

model.setObjective(transport_cost + unmet_cost, GRB.MINIMIZE)

print("\nObjective function set: minimize (transport_cost + penalty * unmet_demand)")

# Constraints
print("\nAdding constraints...")

# 1. Demand satisfaction
for j in deficit_ids:
    model.addConstr(
        gp.quicksum(x[i, j] for i in excess_ids) + u[j] == demand[j],
        name=f"demand_{j}"
    )

print(f"  Added {len(deficit_ids)} demand constraints")

# 2. Supply limits
for i in excess_ids:
    model.addConstr(
        gp.quicksum(x[i, j] for j in deficit_ids) <= supply[i],
        name=f"supply_{i}"
    )

print(f"  Added {len(excess_ids)} supply constraints")

# Solve
print("\n" + "="*60)
print("SOLVING OPTIMIZATION MODEL")
print("="*60 + "\n")

model.optimize()

if model.status == GRB.OPTIMAL:
    print("\n" + "="*60)
    print("✅ OPTIMAL SOLUTION FOUND")
    print("="*60)
    
    # Extract flows
    flows = []
    for i in excess_ids:
        for j in deficit_ids:
            val = x[i, j].X
            if val > 1e-6:
                flows.append({
                    "from_pantry": i,
                    "to_pantry": j,
                    "lbs": val,
                    "distance_miles": dist_df_pantry.loc[i, j]
                })
    
    flows_df = pd.DataFrame(flows)
    
    # Extract unmet demand
    unmet_demand = {j: u[j].X for j in deficit_ids}
    total_unmet = sum(unmet_demand.values())
    
    # Calculate statistics
    total_transported = flows_df['lbs'].sum() if len(flows_df) > 0 else 0
    total_distance = (flows_df['lbs'] * flows_df['distance_miles']).sum() if len(flows_df) > 0 else 0
    avg_distance = flows_df['distance_miles'].mean() if len(flows_df) > 0 else 0
    
    print(f"\n📊 SOLUTION STATISTICS:")
    print(f"  Total flows created: {len(flows)}")
    print(f"  Total food transported: {total_transported:,.2f} lbs")
    print(f"  Total unmet demand: {total_unmet:,.2f} lbs ({total_unmet/sum(demand.values())*100:.1f}% of total demand)")
    print(f"  Total transport cost: {total_distance:,.2f} lb-miles")
    print(f"  Average transport distance: {avg_distance:.2f} miles")
    print(f"  Objective value: {model.objVal:,.2f}")
    
    # Save results
    if len(flows_df) > 0:
        flows_df.to_csv("data/model/optimal_flows.csv", index=False)
        print(f"\n Results saved to data/model/optimal_flows.csv")
        
        print("\n Sample flows:")
        print(flows_df.head(10).to_string(index=False))
    else:
        print("\n  No flows generated (all demand is unmet)")
    
    # Show pantries with highest unmet demand
    if total_unmet > 0:
        unmet_df = pd.DataFrame([
            {"pantry_id": j, "unmet_lbs": unmet_demand[j], "original_demand": demand[j]}
            for j in deficit_ids if unmet_demand[j] > 1
        ]).sort_values("unmet_lbs", ascending=False)
        
        print(f"\n Top 10 pantries with unmet demand:")
        print(unmet_df.head(10).to_string(index=False))

elif model.status == GRB.INFEASIBLE:
    print("\n MODEL IS INFEASIBLE")
    print("Computing IIS (Irreducible Inconsistent Subsystem)...")
    model.computeIIS()
    model.write("model_iis.ilp")
    print("IIS written to model_iis.ilp")
    
elif model.status == GRB.UNBOUNDED:
    print("\n MODEL IS UNBOUNDED")
    
else:
    print(f"\n Optimization failed with status code: {model.status}")
    print("See Gurobi documentation for status code meanings")

Loaded 197 neighborhoods and 515 pantries

Excess NTAs: 55
Deficit NTAs: 142
Excess pantries: 231
Deficit pantries: 283

Supply pantries with data: 231
Demand pantries with data: 283
Total supply (lbs): 50,589,455.30
Total demand (lbs): 96,655,913.95
Excess pantries with valid coords: 231
Deficit pantries with valid coords: 283

Computing distance matrix...
  Processing excess pantry 1/231
  Processing excess pantry 51/231
  Processing excess pantry 101/231
  Processing excess pantry 151/231
  Processing excess pantry 201/231

Distance matrix shape: (231, 283)
Distance range: 0.04 to 33.10 miles
Supply dict keys: 231
Demand dict keys: 283
Distance matrix rows (excess): 231
Distance matrix columns (deficit): 283
All supply keys in distance index: True
All demand keys in distance columns: True

Total supply available: 50,589,455.30 lbs
Total demand needed: 96,655,913.95 lbs
Supply/Demand ratio: 52.34%

BUILDING OPTIMIZATION MODEL
Set parameter OutputFlag to value 1

Variables created:
  

In [ ]:
print("\n" + "="*60)
print("SENSITIVITY ANALYSIS: Finding Optimal Penalty")
print("="*60)

lambda_values = [1, 5, 10, 20, 50, 100]
results = []

for lam in lambda_values:
    print(f"\nTesting λ = {lam}...")
    
    # Build model
    model = gp.Model("test")
    model.setParam("OutputFlag", 0)  # Quiet mode
    
    # Variables
    x = {(i, j): model.addVar(lb=0.0) for i in excess_ids for j in deficit_ids}
    u = {j: model.addVar(lb=0.0) for j in deficit_ids}
    model.update()
    
    # Objective
    transport = gp.quicksum(dist_df_pantry.loc[i, j] * x[i, j] 
                            for i in excess_ids for j in deficit_ids)
    unmet = lam * gp.quicksum(u[j] for j in deficit_ids)
    model.setObjective(transport + unmet, GRB.MINIMIZE)
    
    # Constraints
    for j in deficit_ids:
        model.addConstr(gp.quicksum(x[i, j] for i in excess_ids) + u[j] == demand[j])
    for i in excess_ids:
        model.addConstr(gp.quicksum(x[i, j] for j in deficit_ids) <= supply[i])
    
    # Solve
    model.optimize()
    
    if model.status == GRB.OPTIMAL:
        transported = sum(x[i,j].X for i in excess_ids for j in deficit_ids)
        total_unmet = sum(u[j].X for j in deficit_ids)
        transport_cost = sum(dist_df_pantry.loc[i,j] * x[i,j].X 
                            for i in excess_ids for j in deficit_ids)
        avg_dist = transport_cost / transported if transported > 0 else 0
        
        results.append({
            'lambda': lam,
            'transported_lbs': transported,
            'unmet_lbs': total_unmet,
            'pct_demand_met': transported / sum(demand.values()) * 100,
            'avg_distance': avg_dist,
            'total_lb_miles': transport_cost
        })

# Display results
results_df = pd.DataFrame(results)
print("\n" + "="*60)
print("SENSITIVITY ANALYSIS RESULTS")
print("="*60)
print(results_df.to_string(index=False))

# Save
results_df.to_csv("data/model/sensitivity_analysis.csv", index=False)
print("\n Sensitivity analysis saved")

# Find the "elbow" - best balance
results_df['efficiency'] = results_df['transported_lbs'] / results_df['total_lb_miles']
best_lambda = results_df.loc[results_df['efficiency'].idxmax(), 'lambda']
print(f"\n Recommended λ value: {best_lambda} (best efficiency)")


SENSITIVITY ANALYSIS: Finding Optimal Penalty

Testing λ = 1...

Testing λ = 5...

Testing λ = 10...

Testing λ = 20...

Testing λ = 50...

Testing λ = 100...

SENSITIVITY ANALYSIS RESULTS
 lambda  transported_lbs    unmet_lbs  pct_demand_met  avg_distance  total_lb_miles
      1     1.138586e+07 8.527006e+07       11.779784      0.605499    6.894122e+06
      5     4.284381e+07 5.381211e+07       44.326108      2.190614    9.385424e+07
     10     4.948184e+07 4.717407e+07       51.193806      2.794067    1.382556e+08
     20     5.058946e+07 4.606646e+07       52.339741      2.961480    1.498197e+08
     50     5.058946e+07 4.606646e+07       52.339741      2.961480    1.498197e+08
    100     5.058946e+07 4.606646e+07       52.339741      2.961480    1.498197e+08

 Sensitivity analysis saved

 Recommended λ value: 1 (best efficiency)


In [ ]:
print("\n" + "="*60)
print("SOLUTION BREAKDOWN BY BOROUGH")
print("="*60)

# Add borough info to pantries
borough_map = {
    'BX': 'Bronx', 'BK': 'Brooklyn', 'MN': 'Manhattan',
    'QN': 'Queens', 'SI': 'Staten Island'
}

excess_pantries['borough'] = excess_pantries['nta2020_clean'].str[:2].map(borough_map)
deficit_pantries['borough'] = deficit_pantries['nta2020_clean'].str[:2].map(borough_map)

if len(flows_df) > 0:
    # Merge borough info
    flows_df = flows_df.merge(
        excess_pantries[['id', 'borough']].rename(columns={'borough': 'from_borough'}),
        left_on='from_pantry', right_on='id', how='left'
    ).merge(
        deficit_pantries[['id', 'borough']].rename(columns={'borough': 'to_borough'}),
        left_on='to_pantry', right_on='id', how='left'
    )
    
    # Summary by borough
    borough_summary = flows_df.groupby(['from_borough', 'to_borough']).agg({
        'lbs': 'sum',
        'distance_miles': 'mean',
        'from_pantry': 'count'
    }).rename(columns={'from_pantry': 'num_flows'}).reset_index()
    
    borough_summary['lbs'] = borough_summary['lbs'].round(0)
    borough_summary['distance_miles'] = borough_summary['distance_miles'].round(2)
    
    print("\n🗺️  Food flows by borough pair:")
    print(borough_summary.sort_values('lbs', ascending=False).to_string(index=False))
    
    # Within vs between borough flows
    flows_df['flow_type'] = flows_df.apply(
        lambda x: 'Within borough' if x['from_borough'] == x['to_borough'] else 'Between boroughs',
        axis=1
    )
    
    flow_type_summary = flows_df.groupby('flow_type').agg({
        'lbs': 'sum',
        'distance_miles': 'mean'
    }).round(2)
    
    print("\n🏙️  Within vs Between Borough:")
    print(flow_type_summary)
    
    # Supply and demand by borough
    supply_by_boro = excess_pantries.groupby('borough').agg({
        'id': 'count',
        'nta2020_clean': lambda x: x.map(lambda n: supply.get(
            excess_pantries[excess_pantries['nta2020_clean']==n]['id'].iloc[0] if len(excess_pantries[excess_pantries['nta2020_clean']==n]) > 0 else '0', 0
        )).sum()
    }).rename(columns={'id': 'num_pantries', 'nta2020_clean': 'total_supply'})
    
    print("\n📦 Supply by borough:")
    print(supply_by_boro)


SOLUTION BREAKDOWN BY BOROUGH

🗺️  Food flows by borough pair:
 from_borough    to_borough       lbs  distance_miles  num_flows
     Brooklyn      Brooklyn 4083459.0            0.47         62
        Bronx         Bronx 2940846.0            0.53         33
       Queens        Queens 2810463.0            0.62         24
    Manhattan     Manhattan  940694.0            0.38         14
Staten Island Staten Island  390237.0            0.60          4
    Manhattan         Bronx  204011.0            0.83          3
     Brooklyn        Queens   16148.0            0.68          2

🏙️  Within vs Between Borough:
                          lbs  distance_miles
flow_type                                    
Between boroughs    220158.76            0.77
Within borough    11165698.82            0.51

📦 Supply by borough:
               num_pantries  total_supply
borough                                  
Bronx                    34  5.238664e+06
Brooklyn                 98  2.155641e+07
Manhattan 